Ici on récupère les données via PISTE l'API permettant d'avoir accès aux données de Légifrance, le code de connexion OAuth est donné sur le Gitlab de Piste à cette [adresse](https://gitlab.com/piste_lab/oauth_connectors/-/blob/master/Python/Oauth2ClientCredentialsTest.py?ref_type=heads). Il est tout de même nécessaire d'installer les modules du code suivant.

In [1]:
%pip install python-dotenv
%pip install requests-oauthlib


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from dotenv import load_dotenv
import requests
from requests_oauthlib import OAuth2Session
import requests
import pandas as pd
import math
import json
from datetime import datetime, timedelta
from threading import Thread
import zipfile
import numpy as np 

In [3]:
def get_client():
    """
    Récupère un client OAuth2Session configuré avec un token d'accès depuis le serveur OAuth.

    :return: Un objet OAuth2Session prêt à être utilisé pour des requêtes API.
    """
    TOKEN_URL = "https://oauth.piste.gouv.fr/api/oauth/token"

    # Charger les identifiants client depuis le fichier .env
    load_dotenv()
    client_id = os.getenv("CLIENT_ID")
    client_secret = os.getenv("CLIENT_SECRET")

    # Requête pour obtenir le token
    res = requests.post(
        TOKEN_URL,
        data={
            "grant_type": "client_credentials",
            "client_id": client_id,
            "client_secret": client_secret,
            "scope": "openid"
        }
    )

    res.raise_for_status()  # Lever une erreur si la requête échoue
    token = res.json()

    # Retourner un client OAuth2Session configuré
    return OAuth2Session(client_id, token=token)

In [4]:
def save_results_to_file(results, file_name, current_page):
    """
    Sauvegarde les résultats dans un fichier JSON. Si le fichier existe, ajoute les nouvelles données.

    :param results: Liste des résultats à sauvegarder.
    :param file_name: Nom du fichier JSON.
    :param current_page: La page actuelle traitée.
    """
    try:
        # Charger les données existantes si le fichier existe
        with open(file_name, "r", encoding="utf-8") as file:
            existing_data = json.load(file)
            if not isinstance(existing_data, dict):
                raise ValueError("Le fichier de sauvegarde n'est pas correctement structuré.")
            existing_results = existing_data.get("results", [])
            start_page = existing_data.get("current_page", 1)
    except (FileNotFoundError, ValueError):
        existing_results = []
        start_page = 1

    # Ajouter les nouveaux résultats
    existing_results.extend(results)

    # Sauvegarder les résultats mis à jour avec la page actuelle
    with open(file_name, "w", encoding="utf-8") as file:
        json.dump({"results": existing_results, "current_page": current_page}, file, ensure_ascii=False, indent=4)

In [5]:
def collect_all_results(api_host, code):
    # Nom unique pour le fichier de sauvegarde
    file_name = "results.json"

    # Charger la page de démarrage depuis le fichier de sauvegarde
    try:
        with open(file_name, "r", encoding="utf-8") as file:
            saved_data = json.load(file)
            if not isinstance(saved_data, dict):
                raise ValueError("Le fichier de sauvegarde n'est pas correctement structuré.")
            start_page = saved_data.get("current_page", 1)
    except (FileNotFoundError, ValueError):
        print(f"Fichier {file_name} introuvable ou mal structuré. Démarrage depuis la première page.")
        start_page = 1

    # Initialiser le client OAuth2Session et gérer le renouvellement
    client = get_client()
    expires_in = 55*60
    token_expiry = datetime.now() + timedelta(seconds=expires_in)  # Renouveler 5 minute avant expiration

    # Récupérer le total de résultats et calculer le nombre de pages
    response = client.post(api_host, json=code).json()
    total_results = response.get("totalResultNumber", 0)
    page_size = code["recherche"]["pageSize"]
    total_pages = math.ceil(total_results / page_size)

    print(f"Total de résultats : {total_results}")
    print(f"Nombre de pages à récupérer : {total_pages}")
    print(f"Reprise à partir de la page {start_page}")

    # Liste pour stocker les résultats courants
    all_results = []

    for page_number in range(start_page, total_pages + 1):
        # Vérifier si le token doit être renouvelé
        if datetime.now() >= token_expiry:
            print("Renouvellement du client OAuth...")
            client = get_client()
            token_expiry = datetime.now() + timedelta(seconds=expires_in)

        print(f"Récupération de la page {page_number}/{total_pages}...")
        code["recherche"]["pageNumber"] = page_number
        response = client.post(api_host, json=code).json()
        page_results = response.get("results", [])

        if response.get("error") == 503:
            print(response)
            break

        if page_number % 10 == 0: 
            print(response)

        # Ajouter les résultats de la page courante
        all_results.extend(page_results)

        # Sauvegarder les résultats toutes les 20 pages ou à la dernière page
        if page_number % 20 == 0 or page_number == total_pages:
            print(f"Ajout des pages jusqu'à la page {page_number} dans {file_name}...")
            save_results_to_file(all_results, file_name, page_number)

            # Réinitialiser la liste des résultats sauvegardés
            all_results = []

    print(f"Récupération terminée. Dernière page sauvegardée : {total_pages}")
    return total_pages


In [6]:
API_HOST = "https://api.piste.gouv.fr/dila/legifrance/lf-engine-app"

code_api_LODA = {
  "recherche": {
    "filtres": [
      {
        "dates": {
          "start": "1996-01-01",
          "end": "2022-08-31"
        },
        "facette": "DATE_SIGNATURE"
      }
    ],
    "sort": "SIGNATURE_DATE_DESC",
    "fromAdvancedRecherche": True,
    "champs": [
      {
        "criteres": [
          {
            "valeur": "délinquance",
            "operateur": "OU",
            "typeRecherche": "UN_DES_MOTS"
          },
          {
            "valeur": "crime",
            "operateur": "OU",
            "typeRecherche": "UN_DES_MOTS"
          },
          {
            "valeur": "délit",
            "operateur": "OU",
            "typeRecherche": "UN_DES_MOTS"
          },
          {
            "valeur": "Homicides",
            "operateur": "OU",
            "typeRecherche": "UN_DES_MOTS"
          },
          {
            "valeur": "Vols",
            "operateur": "OU",
            "typeRecherche": "UN_DES_MOTS"
          },
          {
            "valeur": "Stupéfiants",
            "operateur": "OU",
            "typeRecherche": "UN_DES_MOTS"
          },
          {
            "valeur": "Escroquerie",
            "operateur": "OU",
            "typeRecherche": "UN_DES_MOTS"
          },
          {
            "valeur": "Contrefaçon",
            "operateur": "OU",
            "typeRecherche": "UN_DES_MOTS"
          },
          {
            "valeur": "Sequestrations",
            "operateur": "OU",
            "typeRecherche": "UN_DES_MOTS"
          },
          {
            "valeur": "Recels",
            "operateur": "OU",
            "typeRecherche": "UN_DES_MOTS"
          },
          {
            "valeur": "Proxénétisme",
            "operateur": "OU",
            "typeRecherche": "UN_DES_MOTS"
          },
          {
            "valeur": "Menaces",
            "operateur": "OU",
            "typeRecherche": "UN_DES_MOTS"
          },
          {
            "valeur": "Cambriolages",
            "operateur": "OU",
            "typeRecherche": "UN_DES_MOTS"
          },
          {
            "valeur": "infraction",
            "operateur": "OU",
            "typeRecherche": "UN_DES_MOTS"
          },
          {
            "valeur": "Attentats",
            "operateur": "OU",
            "typeRecherche": "UN_DES_MOTS"
          },
          {
            "valeur": "dégradations",
            "operateur": "OU",
            "typeRecherche": "UN_DES_MOTS"
          },
          {
            "valeur": "Outrages",
            "operateur": "OU",
            "typeRecherche": "UN_DES_MOTS"
          }
        ],
        "operateur": "OU",
        "typeChamp": "ALL"
      }
    ],
    "pageSize": 1,
    "pageNumber": 100,
    "operateur": "ET",
    "typePagination": "DEFAUT"
  },
  "fond":"LODA_DATE"
}

In [7]:
API_HOST = "https://api.piste.gouv.fr/dila/legifrance/lf-engine-app"

code_api = {
    "recherche": {
        "champs": [
            {
                "typeChamp": "ALL",
                "criteres": [
                    {"typeRecherche": "UN_DES_MOTS", "valeur": "délinquance", "operateur": "OU"},
                    {"typeRecherche": "UN_DES_MOTS", "valeur": "crime", "operateur": "OU"},
                    {"typeRecherche": "UN_DES_MOTS", "valeur": "délit", "operateur": "OU"},
                    {"typeRecherche": "UN_DES_MOTS", "valeur": "Homicides", "operateur": "OU"},
                    {"typeRecherche": "UN_DES_MOTS", "valeur": "Vols", "operateur": "OU"},
                    {"typeRecherche": "UN_DES_MOTS", "valeur": "Stupéfiants", "operateur": "OU"},
                    {"typeRecherche": "UN_DES_MOTS", "valeur": "Escroquerie", "operateur": "OU"},
                    {"typeRecherche": "UN_DES_MOTS", "valeur": "Contrefaçon", "operateur": "OU"},
                    {"typeRecherche": "UN_DES_MOTS", "valeur": "Sequestrations", "operateur": "OU"},
                    {"typeRecherche": "UN_DES_MOTS", "valeur": "Recels", "operateur": "OU"},
                    {"typeRecherche": "UN_DES_MOTS", "valeur": "Proxénétisme", "operateur": "OU"},
                    {"typeRecherche": "UN_DES_MOTS", "valeur": "Menaces", "operateur": "OU"},
                    {"typeRecherche": "UN_DES_MOTS", "valeur": "Cambriolages", "operateur": "OU"},
                    {"typeRecherche": "UN_DES_MOTS", "valeur": "infraction", "operateur": "OU"},
                    {"typeRecherche": "UN_DES_MOTS", "valeur": "Attentats", "operateur": "OU"},
                    {"typeRecherche": "UN_DES_MOTS", "valeur": "dégradations", "operateur": "OU"},
                    {"typeRecherche": "UN_DES_MOTS", "valeur": "Outrages", "operateur": "OU"}
                ],
                "operateur": "OU"
            }
        ],
        "pageNumber": 1,
        "pageSize": 1,
        "operateur": "ET",
        "sort": "DATE",
        "typePagination": "DEFAUT"
    },
    "fond": "ALL"
}


In [ ]:
results = collect_all_results(API_HOST+"/search", code_api)

Total de résultats : 278632
Nombre de pages à récupérer : 278632
Reprise à partir de la page 45740
Récupération de la page 45740/278632...
{'executionTime': 1004, 'results': [{'titles': [{'id': 'LEGITEXT000006070719', 'cid': 'LEGITEXT000006070719', 'title': 'Code pénal', 'legalStatus': None, 'startDate': None, 'endDate': None, 'nature': None}], 'text': None, 'type': '_doc', 'nature': 'CODE', 'origin': 'LEGI', 'etat': 'VIGUEUR', 'date': '1990-09-01T00:00:00.000+0000', 'sections': [{'id': 'LEGISCTA000006165338', 'title': 'Section 1 : Du <mark>recel</mark>', 'dateVersion': '2002-01-01T00:00:00.000+0000', 'legalStatus': 'VIGUEUR', 'extracts': [{'id': 'LEGIARTI000006418234', 'title': '321-1', 'legalStatus': 'VIGUEUR', 'dateVersion': '2002-01-01T00:00:00.000+0000', 'dateDebut': '2002-01-01T00:00:00.000+0000', 'dateFin': '2999-01-01T00:00:00.000+0000', 'searchFieldName': None, 'num': '321-1', 'values': ["[...] Le <mark>recel</mark> est le fait de dissimuler, de détenir ou de transmettre une c

In [ ]:
def zipper_fichier(fichier, zip_nom):
    """
    Crée un fichier ZIP contenant le fichier spécifié et supprime le fichier d'origine

    :param fichier: Chemin du fichier à zipper.
    :param zip_nom: Nom du fichier ZIP de sortie.
    """
    with zipfile.ZipFile(zip_nom, 'w') as zipf:
        zipf.write(fichier, arcname=fichier.split('/')[-1]) 
        os.remove(fichier)

In [ ]:
def extraire_json_du_zip(fichier_zip, fichier_sortie):
    """
    Extrait un fichier JSON contenu dans une archive ZIP et le sauvegarde.

    :param fichier_zip: Chemin de l'archive ZIP contenant le fichier JSON.
    :param fichier_sortie: Chemin du fichier JSON extrait.
    """
    with zipfile.ZipFile(fichier_zip, 'r') as zipf:
        json_fichier = zipf.namelist()[0]  
        with zipf.open(json_fichier) as file:
            data = json.load(file)
        with open(fichier_sortie, 'w', encoding='utf-8') as json_file:
            json.dump(data, json_file, indent=4)

In [ ]:
zipper_fichier('/home/onyxia/work/Python-pour-la-data-science-2A/data/results.json', 'results')

In [ ]:
extraire_json_du_zip('/home/onyxia/work/Python-pour-la-data-science-2A/data/results.zip', '/home/onyxia/work/Python-pour-la-data-science-2A/data/results.json')

In [ ]:
pwd

In [ ]:
def remaining_page_number():
    api_host = API_HOST+"/search"
    client = get_client()
    response = client.post(api_host, json=code_api).json()
    total_results = response.get("totalResultNumber", 0)
    try:
        # Charger les données existantes si le fichier existe
        with open("results.json", "r", encoding="utf-8") as file:
            existing_data = json.load(file)
            if not isinstance(existing_data, dict):
                raise ValueError("Le fichier de sauvegarde n'est pas correctement structuré.")
            start_page = existing_data.get("current_page", 1)
    except (FileNotFoundError, ValueError):
        start_page = 0
    remaining_page = total_results-start_page
    return remaining_page    

In [ ]:
def collect_all_results_between(api_host, code, page_to_start, page_to_end, thread_number):
    client = get_client()
    expires_in = 55*60
    token_expiry = datetime.now() + timedelta(seconds=expires_in)

    file_name = str(thread_number)+"results.json"


    all_results = []

    for page_number in range(page_to_start, page_to_end + 1):
        # Vérifier si le token doit être renouvelé
        if datetime.now() >= token_expiry:
            print("Renouvellement du client OAuth...")
            client = get_client()
            token_expiry = datetime.now() + timedelta(seconds=expires_in)

        print(f"Récupération de la page {page_number}/{page_to_end - page_to_start +1}...")
        code["recherche"]["pageNumber"] = page_number
        response = client.post(api_host, json=code).json()
        page_results = response.get("results", [])

        if response.get("error") == 503:
            print(response)
            break

        if page_number % 10 == 0: 
            print(response)

        # Ajouter les résultats de la page courante
        all_results.extend(page_results)

        # Sauvegarder les résultats toutes les 20 pages ou à la dernière page
        if page_number % 20 == 0 or page_number == page_to_end:
            print(f"Ajout des pages jusqu'à la page {page_number} dans {file_name}...")
            save_results_to_file(all_results, file_name, page_number)

            # Réinitialiser la liste des résultats sauvegardés
            all_results = []


In [ ]:
def generate_functions(n):
        api_host = API_HOST+"/search"
        client = get_client()
        remaining_page = remaining_page_number()
        functions = {}
        response = client.post(API_HOST+"/search", json=code_api).json()
        total_results = response.get("totalResultNumber", 0)
        try:
            with open("results.json", "r", encoding="utf-8") as file:
                existing_data = json.load(file)
                if not isinstance(existing_data, dict):
                    raise ValueError("Le fichier de sauvegarde n'est pas correctement structuré.")
                start_page = existing_data.get("current_page", 1)
        except (FileNotFoundError, ValueError):
            start_page = 1

        page_state = [start_page, start_page]

        for i in range(1, n + 1):
            if i != n : 
                    page_to_end = [int(np.floor(remaining_page*i/n)+ page_state[1])]
            else : 
                    page_to_end = [total_results]

            thread_number = i

            def func_template(idx=i, start=page_state[0], end=page_to_end[0], thread_nbr= thread_number):
                collect_all_results_between(API_HOST+"/search", code_api, start, end, thread_nbr)

            page_state[0]= page_to_end[0] + 1
                
            functions[f"f_{i}"] = func_template
        return functions

In [ ]:
def functions_to_thread(n):
    generated_functions = generate_functions(n)
    functions = [(generated_functions[f"f_{i}"], [], {}) for i in range(1, n+1) ]
    return functions

In [ ]:
r = functions_to_thread(4)
r

In [ ]:
k = generate_functions(4)
print(k)
k[f"f_{3}"]

In [ ]:
def run_in_threads(functions):
    """
    Lance un ensemble de fonctions dans des threads séparés.

    :param functions: Liste de tuples (fonction, args, kwargs) où :
                      - `fonction` est la fonction à exécuter
                      - `args` est une liste des arguments positionnels
                      - `kwargs` est un dictionnaire des arguments nommés
    """
    threads = []

    for func, args, kwargs in functions:
        thread = Thread(target=func, args=args, kwargs=kwargs)
        threads.append(thread)
        thread.start()

    # Attendre la fin de tous les threads
    for thread in threads:
        thread.join()

In [ ]:
l = run_in_threads(functions_to_thread(8))

In [ ]:
def segmenter_json_par_parties(fichier_json, dossier_sortie, nombre_parties):
    """
    Segmente un fichier JSON en un nombre spécifique de parties.

    :param fichier_json: Chemin du fichier JSON à segmenter.
    :param dossier_sortie: Dossier où les segments seront sauvegardés.
    :param nombre_parties: Nombre de parties dans lesquelles le fichier sera segmenté.
    """
    # Lire le contenu du fichier JSON
    with open(fichier_json, 'r', encoding='utf-8') as file:
        data = json.load(file)

    # Calculer la taille approximative de chaque partie
    taille_segment = math.ceil(len(data) / nombre_parties)

    # Créer le dossier de sortie s'il n'existe pas
    os.makedirs(dossier_sortie, exist_ok=True)

    # Segmenter les données
    for i in range(0, len(data), taille_segment):
        segment = data[i:i + taille_segment]
        segment_path = os.path.join(dossier_sortie, f'segment_{i // taille_segment + 1}.json')
        
        # Écrire chaque segment dans un fichier
        with open(segment_path, 'w', encoding='utf-8') as segment_file:
            json.dump(segment, segment_file, indent=4)
    
    print(f"Fichier JSON segmenté en {nombre_parties} parties dans le dossier '{dossier_sortie}'.")


In [ ]:
segmenter_json_par_parties("/home/onyxia/work/Python-pour-la-data-science-2A/data/results.json", "data", 3 )

In [ ]:
def assembler_json(dossier_segments, fichier_sortie):
    """
    Assemble plusieurs fichiers JSON en un seul fichier et supprime les segments.

    :param dossier_segments: Dossier contenant les segments JSON.
    :param fichier_sortie: Chemin du fichier JSON de sortie.
    """
    fichiers = sorted(os.listdir(dossier_segments))  # Trier les segments par nom
    data_combinee = []

    for fichier in fichiers:
        segment_path = os.path.join(dossier_segments, fichier)
        with open(segment_path, 'r', encoding='utf-8') as segment_file:
            data_combinee.extend(json.load(segment_file))

    # Écrire les données combinées dans un seul fichier JSON
    with open(fichier_sortie, 'w', encoding='utf-8') as output_file:
        json.dump(data_combinee, output_file, indent=4)
    
    print(f"Segments JSON assemblés dans '{fichier_sortie}', et les segments supprimés.")

Pour pour envoyer des requêtes à l'API nous devons d'abord obtenir un token, après s'être inscrit sur le [site de PISTE](https://piste.gouv.fr/). Ce token est obtenu en envoyant une requête contenant mon identifiant client et mon code de client au site d'autorisation. Nous avons ensuite une autorisation d'une heure avec ce token pour exploiter l'API de Légifrance. 

Maintenant en utilisant le lien d'exploitation de l'API auquel on rajoute l'endpoints qui permet d'accéder à ce que l'on veut faire nous récupérons les données. La liste des endpoints pour l'API de Légifrance est disponible [ici](https://piste.gouv.fr/index.php?option=com_apiportal&view=apitester&usage=api&apitab=tests&apiName=L%C3%A9gifrance&apiId=7e5a0e1d-ffcc-40be-a405-a1a5c1afe950&managerId=3&type=rest&apiVersion=2.4.2&Itemid=179&swaggerVersion=2.0&lang=fr).

Le code ci-dessous permet de récupérer un ensemble de documents appartenant à l'ensemble des lois, ordonnance, décrets et arrêtés de Légifrance entre le premier janvier 1996 et le 31 août 2022 qui contiennent au moins un des mots d'une liste définie. Cette liste comprend les termes des taux de délinquances détaillés [ici](database_délinquance.ipynb#Calcul-des-taux-de-délinquance). Ce code est un json et sera envoyé par une méthode post à l'API, elle nous renverra un nombre de résultats limités (100 ici). Ce code est très inspiré de celui disponible sur le [site de l'API](https://piste.gouv.fr/index.php?option=com_apiportal&view=apitester&usage=api&apitab=tests&apiName=L%C3%A9gifrance&apiId=7e5a0e1d-ffcc-40be-a405-a1a5c1afe950&managerId=3&type=rest&apiVersion=2.4.2&Itemid=179&swaggerVersion=2.0&lang=fr) à l'endpoint /search.

On définit une fonction qui nous permet de récupérer tous les documents de notre recherche, car chaque recherche est limitée à une page de 100 éléments.

In [ ]:

import pandas as pd

def results_to_dataframe(json_data):
    data = []
    
    # Accéder à la liste des résultats
    results = json_data.get("results", [])
    
    for result in results:
        
        # Extraire les informations requises
        titles = result.get("titles")
        title_info = titles[0] if isinstance(titles, list) and titles else {}
        
        title = title_info.get("title")
        id = title_info.get("id")
        date = result.get("date")
        nature = result.get("nature")
        etat = result.get("etat")
        origin = result.get("origin")
        date_publication = result.get("datePublication")
        
        # Ajouter les données dans la liste
        data.append({
            "Titre": title,
            "ID": id,
            "Date": date,
            "Nature": nature,
            "Etat": etat,
            "Origine": origin,
            "Date Publication": date_publication
        })
    
    # Créer et retourner un DataFrame
    return pd.DataFrame(data)


In [ ]:
import pandas as pd

def results_to_dataframe(json_data):
    data = []
    
    # Accéder à la liste des résultats
    results = json_data.get("results", [])
    
    if not isinstance(results, list):
        raise ValueError("La clé 'results' doit contenir une liste.")
    
    for result in results:
        if not isinstance(result, dict):
            continue  # Ignorer les éléments non dictionnaires
        
        # Extraire les informations requises
        titles = result.get("titles")
        title_info = titles[0] if isinstance(titles, list) and titles else {}
        
        if not isinstance(title_info, dict):
            title_info = {}
        
        title = title_info.get("title")
        id = title_info.get("id")
        date = result.get("date")
        nature = result.get("nature")
        etat = result.get("etat")
        origin = result.get("origin")
        date_publication = result.get("datePublication")
        
        # Ajouter les données dans la liste
        data.append({
            "Titre": title,
            "ID": id,
            "Date": date,
            "Nature": nature,
            "Etat": etat,
            "Origine": origin,
            "Date Publication": date_publication
        })
    
    # Créer et retourner un DataFrame
    return pd.DataFrame(data)

# Exemple d'utilisation
# results_to_dataframe(votre_json_data)


In [ ]:
with open("results_LODA.json", "r", encoding="utf-8") as file:
    json_data = json.load(file)
df_LODA = results_to_dataframe(json_data)

In [ ]:
df_LODA = df_LODA.drop_duplicates()
df_LODA.to_csv("resultats_legifrance_loda.csv", index=False)